In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

# Data Acquisition

In [ ]:
movies_file_path = "tmdb_5000_movies.csv"
movies = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "tmdb/tmdb-movie-metadata",
  movies_file_path
)

credits_file_path = "tmdb_5000_credits.csv"
credits = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "tmdb/tmdb-movie-metadata",
  credits_file_path
)

In [ ]:
# merge both dataframs into one
movies = movies.merge(credits, on="title")

In [ ]:
movies.info()

In [ ]:
movies = movies[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

# Preprocessing

In [ ]:
# check for null values
movies.isnull().sum()
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
# check and remove duplicates if any
movies.duplicated().sum()

In [ ]:
movies.iloc[1].genres
movies.iloc[1].keywords
# movies.iloc[1].cast

In [ ]:
# make proper structure of genres, keyword, cast, crew

# helper function
import ast
def convert(obj):
    list = []
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list

def fetch_director(obj):
  list = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      list.append(i['name'])
      break
  return list


In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
# remove white spaces
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

In [ ]:
# combine columns to one
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
# getting our new df
new_df = movies[['id', 'title', 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

## Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

# Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

# Model

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]


  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')